# Part 2 Solution - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - Some of the options require a Mapbox API token:
    - However, we will be using the options that DO NOT require a token.
        - https://studio.mapbox.com/

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [3]:
## Load in csv.gz
df = pd.read_csv('../Data/Seattle-pizza.csv.gz')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","['pickup', 'delivery']",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",['delivery'],$$,"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011
2,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],NaN,"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113
3,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...","['pickup', 'delivery']",$,"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405
4,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-aZrUu...,False,https://www.yelp.com/biz/italian-family-pizza-...,1009,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 47.60937, 'longitude': -122.32546}","['pickup', 'delivery']",$$,"{'address1': '1028 Madison St', 'address2': No...",1.206538e+10,(206) 538-0040,2271.307105


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- 2. We also want to be able to show the restaurants:
    - name
    - price
    - type of transactions (pickup/delivery/restaurant reservation)
    - address

### Separating Latitude and Longitude

In [4]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 47.6146934, 'longitude': -122.312..."
1,"{'latitude': 47.5976491915013, 'longitude': -1..."
2,"{'latitude': 47.608127, 'longitude': -122.302435}"
3,"{'latitude': 47.615379179632, 'longitude': -12..."
4,"{'latitude': 47.60937, 'longitude': -122.32546}"
...,...
834,"{'latitude': 47.6157033283809, 'longitude': -1..."
835,"{'latitude': 47.6759, 'longitude': -122.30461}"
836,"{'latitude': 47.61252119845868, 'longitude': -..."
837,"{'latitude': 47.615402, 'longitude': -122.2039..."


- Why didn't that work???

In [5]:
## slice out a single test coordinate
test_coord = df.loc[1, 'coordinates']
test_coord

"{'latitude': 47.5976491915013, 'longitude': -122.313305988361}"

In [6]:
type(test_coord)

str

- Its not a dictionary anymore!!! What??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our `string dictionary` into an `actual dictionary`. 

In [7]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

### JSON requires double quotes!
We got a `JSON Decode Error` because JSON was `expecting double quotes` inside
of the dictionary

In [8]:
# Check the single record
test_coord

"{'latitude': 47.5976491915013, 'longitude': -122.313305988361}"

### We are now going to use the .replace( ) function to replace single ' with double "

In [9]:
## replace single ' with double " 
test_coord = test_coord.replace("'", '"')
test_coord

'{"latitude": 47.5976491915013, "longitude": -122.313305988361}'

In [10]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 47.5976491915013, 'longitude': -122.313305988361}

In [11]:
# viewing type after using json.loads
type(json.loads(test_coord))

dict

### Now, how can we apply this same process to the entire column??

In [12]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'", '"')
## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

In [13]:
## slice out a single test coordinate
test_coord = df.loc[5, 'coordinates']
test_coord

{'latitude': 47.61849, 'longitude': -122.31664}

In [14]:
type(test_coord)

dict

### Using .apply with pd.Series to convert a dictionary column into multiple columns
This is the process of unpacking the dictionary to columns

In [15]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,latitude,longitude
0,47.614693,-122.312764
1,47.597649,-122.313306
2,47.608127,-122.302435
3,47.615379,-122.323231
4,47.609370,-122.325460
...,...,...
834,47.615703,-122.200735
835,47.675900,-122.304610
836,47.612521,-122.201333
837,47.615402,-122.203978


In [16]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df, df['coordinates'].apply(pd.Series)], axis = 1)
#df = df.drop(columns = 'coordinates')
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","['pickup', 'delivery']",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,47.614693,-122.312764
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",['delivery'],$$,"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,47.597649,-122.313306


In [17]:
df = df.drop(columns = 'coordinates')
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['pickup', 'delivery']",$$,"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,47.614693,-122.312764
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,47.597649,-122.313306


### Activity -Padlet : Unpacking dictionary


- https://padlet.com/swhaley9/unpacking-dictionaries-d7mlgygcjtm0s4xs

## Creating a Simple Map

- Mapbox API: https://www.mapbox.com/
- Mapbox API Documentation: https://docs.mapbox.com/api/overview/

- Use the plotly express `scatter_mapbox` function

In [20]:
## use scatter_mapbox for map
px.scatter_mapbox(df, lat = 'latitude', lon = 'longitude', mapbox_style = 'open-street-map')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - rating
    - transaction type (delivery/takeout)
    - address
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [22]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(df, lat = 'latitude', lon = 'longitude', mapbox_style = 'open-street-map', 
                 hover_name = 'name', hover_data = ['price', 'rating', 'transactions', 'location'])

### Fixing the Location Column

In [23]:
## slice out a test address
test_addr = df.loc[0, 'location']
test_addr

"{'address1': '1546 15th Ave', 'address2': '', 'address3': '', 'city': 'Seattle', 'zip_code': '98122', 'country': 'US', 'state': 'WA', 'display_address': ['1546 15th Ave', 'Seattle, WA 98122']}"

> Also a string-dictionary...

In [24]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['pickup', 'delivery']",$$,"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,47.614693,-122.312764
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,47.597649,-122.313306
2,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],NaN,"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,47.608127,-122.302435
3,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"['pickup', 'delivery']",$,"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,47.615379,-122.323231
4,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-aZrUu...,False,https://www.yelp.com/biz/italian-family-pizza-...,1009,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""1028 Madison St"", ""address2"": No...",1.206538e+10,(206) 538-0040,2271.307105,47.609370,-122.325460


In [25]:
## apply json.loads
df['location'] = df['location'].apply(json.loads)

JSONDecodeError: Expecting value: line 1 column 62 (char 61)

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [26]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "1546 15th Ave", "address2": "", "address3": "", "city": "Seattle", "zip_code": "98122", "country": "US", "state": "WA", "display_address": ["1546 15th Ave", "Seattle, WA 98122"]}'

In [27]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return 'Error'

In [28]:
## test applying our function
df['location'].apply(fix_address)

0      {'address1': '1546 15th Ave', 'address2': '', ...
1      {'address1': '525 Rainier Ave S', 'address2': ...
2                                                  Error
3      {'address1': '700 E Pine St', 'address2': '', ...
4                                                  Error
                             ...                        
834    {'address1': '600 Bellevue Way NE', 'address2'...
835    {'address1': '2112 NE 65th St', 'address2': ''...
836                                                Error
837    {'address1': '100 Bellevue Sq', 'address2': ''...
838                                                Error
Name: location, Length: 839, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [29]:
### save a new display_location column using our function
df['display_location'] = df['location'].apply(fix_address)

In [30]:
## filter for businesses with display_location == "ERROR"
errors = df[df['display_location'] == 'Error']
errors

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
2,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],NaN,"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,47.608127,-122.302435,Error
4,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/-aZrUu...,False,https://www.yelp.com/biz/italian-family-pizza-...,1009,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""1028 Madison St"", ""address2"": No...",1.206538e+10,(206) 538-0040,2271.307105,47.609370,-122.325460,Error
5,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,47,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,[],NaN,"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,47.618490,-122.316640,Error
6,-FOAQv22SXtSBs7nptI3UA,serious-pie-downtown-seattle-2,Serious Pie Downtown,https://s3-media1.fl.yelpcdn.com/bphoto/Yge-Xa...,False,https://www.yelp.com/biz/serious-pie-downtown-...,4512,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['pickup', 'delivery']",$$,"{""address1"": ""2001 4th Ave"", ""address2"": None,...",1.206839e+10,(206) 838-7388,3348.309276,47.613046,-122.340353,Error
10,n6BLr6spgjVWqzUwmVx0bg,johnny-mos-pizzeria-seattle,Johnny Mo's Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/8IAnDP...,False,https://www.yelp.com/biz/johnny-mos-pizzeria-s...,150,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"['pickup', 'delivery']",$$,"{""address1"": ""3272 Fuhrman Ave E"", ""address2"":...",1.206823e+10,(206) 822-6272,4658.301257,47.651420,-122.320830,Error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,_Vq1fXpIs8-6WQy0GO8tjg,genghis-khan-restaurant-seattle,Genghis Khan Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/zVFs_R...,False,https://www.yelp.com/biz/genghis-khan-restaura...,362,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,"['delivery', 'pickup']",$$,"{""address1"": ""82 Stewart St"", ""address2"": None...",1.206682e+10,(206) 682-3606,3496.786275,47.609954,-122.342099,Error
830,Nihv2RfuTZ4TI8BQl3anmA,qfc-kirkland-4,QFC,https://s3-media2.fl.yelpcdn.com/bphoto/NYz4rH...,False,https://www.yelp.com/biz/qfc-kirkland-4?adjust...,32,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,[],$$,"{""address1"": ""457 Central Way"", ""address2"": """"...",1.425827e+10,(425) 827-2205,10212.263314,47.678048,-122.199308,Error
833,AsgtzyqpdjuPyjaEUVcUXA,jewel-of-india-seattle-3,Jewel Of India,https://s3-media2.fl.yelpcdn.com/bphoto/qJGMnl...,False,https://www.yelp.com/biz/jewel-of-india-seattl...,413,"[{'alias': 'indpak', 'title': 'Indian'}]",3.5,"['delivery', 'pickup']",$$,"{""address1"": ""4735 University Way NE"", ""addres...",1.206524e+10,(206) 523-5275,5825.461180,47.664103,-122.313426,Error
836,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,289,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,47.612521,-122.201333,Error


In [31]:
## slice out a new test address and inspect
test_addr

'{"address1": "1546 15th Ave", "address2": "", "address3": "", "city": "Seattle", "zip_code": "98122", "country": "US", "state": "WA", "display_address": ["1546 15th Ave", "Seattle, WA 98122"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [32]:
## remove any rows where display_location == 'ERROR'
df = df.loc[df['display_location'] != 'Error']
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['pickup', 'delivery']",$$,"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ..."
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ..."
3,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"['pickup', 'delivery']",$,"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ..."
7,GJdmwqqkC0nAQfX1qZM3ew,olympia-pizza-and-spaghetti-house-iii-seattle,Olympia Pizza and Spaghetti House III,https://s3-media1.fl.yelpcdn.com/bphoto/KvTgMA...,False,https://www.yelp.com/biz/olympia-pizza-and-spa...,299,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{""address1"": ""516 15th Ave E"", ""address2"": """",...",1.206329e+10,(206) 329-4500,1732.874383,47.623830,-122.312445,"{'address1': '516 15th Ave E', 'address2': '',..."
8,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,180,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ..."


- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [33]:
## slice out a new test address and inspect
test_addr = df.loc[3, 'display_location']['display_address']
test_addr

['700 E Pine St', 'Seattle, WA 98122']

In [34]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_location'].apply(lambda x: x['display_address'])
df.head(2)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location,display_address
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['pickup', 'delivery']",$$,"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]"
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]"


- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [35]:
## slice out a test_address
test_add = df.loc[339, 'display_address']
test_add

['401 Lenora St', 'Seattle, WA 98121']

In [36]:
## test using .join with a "\n"
'\n'.join(test_add)
print('\n'.join(test_add))

401 Lenora St
Seattle, WA 98121


In [37]:
## apply the join to every row with a lambda
df['address'] = df['display_address'].apply(lambda x: '\n'.join(x))
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location,display_address,address
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['pickup', 'delivery']",$$,"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]","1546 15th Ave\nSeattle, WA 98122"
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],$$,"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]","525 Rainier Ave S\nSeattle, WA 98144"
3,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"['pickup', 'delivery']",$,"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ...","[700 E Pine St, Seattle, WA 98122]","700 E Pine St\nSeattle, WA 98122"
7,GJdmwqqkC0nAQfX1qZM3ew,olympia-pizza-and-spaghetti-house-iii-seattle,Olympia Pizza and Spaghetti House III,https://s3-media1.fl.yelpcdn.com/bphoto/KvTgMA...,False,https://www.yelp.com/biz/olympia-pizza-and-spa...,299,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{""address1"": ""516 15th Ave E"", ""address2"": """",...",1.206329e+10,(206) 329-4500,1732.874383,47.623830,-122.312445,"{'address1': '516 15th Ave E', 'address2': '',...","[516 15th Ave E, Seattle, WA 98112]","516 15th Ave E\nSeattle, WA 98112"
8,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,180,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],$$,"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ...","[4235 E Madison St, Seattle, WA 98112]","4235 E Madison St\nSeattle, WA 98112"


In [38]:
test_add = df.loc[339, 'address']
test_add

'401 Lenora St\nSeattle, WA 98121'

### Lastly, Fixing Transactions Column

In [39]:
# Looking at a test transaction
test_transaction = df.loc[1, 'transactions']
test_transaction

"['delivery']"

In [40]:
type(test_transaction)

str

- This is also a string and needs to be converted to a list.

In [41]:
# Replacing single ' with double "
saved_test = test_transaction.replace("'", '"')
saved_test

'["delivery"]'

In [42]:
# Using json.loads on saved_test
json.loads(saved_test)

['delivery']

In [43]:
# Applying transformations to entire column

# Create a new column where the single quotes are replaced with double quotes
df['transactions_split'] = df['transactions'].str.replace("'", '"')
# Apply json.loads to entire column
df['transactions_split'] = df['transactions_split'].apply(json.loads)
df['transactions_split'].head()

0    [pickup, delivery]
1            [delivery]
3    [pickup, delivery]
7            [delivery]
8            [delivery]
Name: transactions_split, dtype: object

In [44]:
df['transactions_split'].value_counts()

[delivery]                                    178
[delivery, pickup]                            138
[pickup, delivery]                             95
[]                                             95
[pickup]                                        7
[pickup, delivery, restaurant_reservation]      4
[restaurant_reservation]                        2
[restaurant_reservation, delivery, pickup]      1
[restaurant_reservation, delivery]              1
[delivery, restaurant_reservation, pickup]      1
[pickup, restaurant_reservation, delivery]      1
Name: transactions_split, dtype: int64

In [45]:
# Converting transactions column into a one-hot-encoded column
exploded = df.explode('transactions_split')
exploded[['name', 'transactions', 'transactions_split']].head()

,name,transactions,transactions_split
0,Bar Cotto,"['pickup', 'delivery']",pickup
0,Bar Cotto,"['pickup', 'delivery']",delivery
1,Humble Pie,['delivery'],delivery
3,Hot Mama's Pizza,"['pickup', 'delivery']",pickup
3,Hot Mama's Pizza,"['pickup', 'delivery']",delivery


In [46]:
# remove NaNs and find unique values
cols_to_make = exploded['transactions_split'].dropna().unique()
cols_to_make

array(['pickup', 'delivery', 'restaurant_reservation'], dtype=object)

In [47]:
# Using a for loop with .str.contains to create new columns
for col in cols_to_make:
    df[col] = df['transactions'].str.contains(col)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,distance,latitude,longitude,display_location,display_address,address,transactions_split,pickup,delivery,restaurant_reservation
0,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,355,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['pickup', 'delivery']",...,1292.526796,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]","1546 15th Ave\nSeattle, WA 98122","[pickup, delivery]",True,True,False
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,369,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],...,2166.834011,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]","525 Rainier Ave S\nSeattle, WA 98144",[delivery],False,True,False
3,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,969,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"['pickup', 'delivery']",...,2080.359405,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ...","[700 E Pine St, Seattle, WA 98122]","700 E Pine St\nSeattle, WA 98122","[pickup, delivery]",True,True,False
7,GJdmwqqkC0nAQfX1qZM3ew,olympia-pizza-and-spaghetti-house-iii-seattle,Olympia Pizza and Spaghetti House III,https://s3-media1.fl.yelpcdn.com/bphoto/KvTgMA...,False,https://www.yelp.com/biz/olympia-pizza-and-spa...,299,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],...,1732.874383,47.623830,-122.312445,"{'address1': '516 15th Ave E', 'address2': '',...","[516 15th Ave E, Seattle, WA 98112]","516 15th Ave E\nSeattle, WA 98112",[delivery],False,True,False
8,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,180,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],...,2897.252844,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ...","[4235 E Madison St, Seattle, WA 98112]","4235 E Madison St\nSeattle, WA 98112",[delivery],False,True,False


### Final Map

In [48]:
## make ourn final map and save as varaible
pfig = px.scatter_mapbox(df, lat = 'latitude', lon = 'longitude', 
                        mapbox_style = 'open-street-map', hover_name = 'name',
                         hover_data = ['price', 'rating', 'address', 'pickup', 
                                      'delivery', 'restaurant_reservation'],
                         color = 'rating',
                         title = 'Restaurants with Pizza in Seattle, WA')

pfig.show()

### Saving Final Figure

In [49]:
## use fig.write_html to save map
pfig.write_html('final_map.html')